Extracts map metadata and transforms it to csv

In [2]:
import csv 
import pandas as pd
import re
import io
import xml.etree.ElementTree as ET

In [3]:
with open('metadata_all.csv', newline='',mode='r',encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row["snapshotDisplayName"] == "Satellite Derived Coastline":
            mari = row

In [ ]:
with open('nspeatlandsac\Peatland_ACTION_completed_restoration_footprints.xmlv') as file:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row["snapshotDisplayName"] == "Satellite Derived Coastline":
            mari = row

In [4]:
print(mari)

{'snapshotYear': '2023', 'datasetName': 'ukho_satellite_derived_coastine_v1', 'marcText': '"\\n=LDR       nem a22     5u 4500\r\\n=034  1\\$b$dW0033059$eW0010057$fN0542649$gN0505410\r\\n=035  \\\\$a000461\r\\n=110  2\\$aUK Hydrographic Office\r\\n=245  10$aSatellite Derived Coastline\r\\n=246  3\\$aSatellite Derived Coastline\r\\n=255  \\\\$bTransverse Mercator projection$c(W 8°38\'00""--E 1°45\'00""/N 60°51\'00""--N 49°51\'00"")\r\\n=264  \\1$aTaunton, UK$bUK Hydrographic Office$c2023\r\\n=300  \\\\$a1 geospatial database\r\\n=342  00$bdegree$c0.0174532925199433$d0.0174532925199433\r\\n=342  01$aTransverse Mercator$g-2$h49$i400,000$j-100,000$k0.9996012717\r\\n=342  02$aOSGB 1936 / British National Grid$bmetre$c1$d1$pBritish National Grid\r\\n=342  05$aOSGB_1936$qAiry 1830$r6377563.396$s299.3249646\r\\n=352  \\\\$aVector$bPoint$qSHP\r\\n=655  \\7$aDigital maps\r\\n=655  \\7$aDigital maps\r\\n=856  40$uhttps://mapportal.ldls.org.uk/view?\r\\n\r\\n"', 'marcXml': '"<?xml version=\'1.0\' e

In [5]:
marcx = mari['marcXml']
print(marcx)

"<?xml version='1.0' encoding='UTF-8'?>\n<marc:collection xmlns:marc=""http://www.loc.gov/MARC21/slim"" xmlns:xsi=""http://www.w3.org/2001/XMLSchema-instance"" xsi:schemaLocation=""http://www.loc.gov/MARC21/slim http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd""><!--MARC skeleton record generated by GDAS: 2021-03-04T15:42:36.695713--><marc:record><marc:leader>     nem a22     5u 4500</marc:leader><marc:datafield ind1=""1"" ind2="" "" tag=""034""><marc:subfield code=""b""/><marc:subfield code=""d"">W0033059</marc:subfield><marc:subfield code=""e"">W0010057</marc:subfield><marc:subfield code=""f"">N0542649</marc:subfield><marc:subfield code=""g"">N0505410</marc:subfield></marc:datafield><marc:datafield ind1="" "" ind2="" "" tag=""035""><marc:subfield code=""a"">000461</marc:subfield></marc:datafield><marc:datafield ind1=""2"" ind2="" "" tag=""110""><marc:subfield code=""a"">UK Hydrographic Office</marc:subfield></marc:datafield><marc:datafield ind1=""1"" ind2=""0"" tag=""245"">

In [6]:
def cleancsvxml(xmlstr):
    a = xmlstr.replace('""','"')
    a = a[1:]
    a = a[:-1]
    a = a.replace("\\r", "\r")
    a = a.replace("\\n", "\n")
    return a

In [7]:
modsx = mari['modsXml']
nmx = cleancsvxml(modsx)
root = ET.fromstring(nmx)
print(root.tag)
print("********")
print(root.attrib)
print("********")
for child in root:
    print(child.tag, child.attrib)

{http://www.loc.gov/mods/v3}modsCollection
********
{'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.loc.gov/mods/v3 http://www.loc.gov/standards/mods/v3/mods-3-4.xsd'}
********
{http://www.loc.gov/mods/v3}mods {'version': '3.4'}


In [9]:
marcx = mari['marcXml']
mcx = cleancsvxml(marcx)
root = ET.fromstringlist(mcx)
print(root.tag)
print("********")
print(root.attrib)
print("********")
for child in root:
    print(child.tag, child.attrib)

{http://www.loc.gov/MARC21/slim}collection
********
{'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.loc.gov/MARC21/slim http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd'}
********
{http://www.loc.gov/MARC21/slim}record {}


In [8]:
metadf = pd.DataFrame(columns=['Author','Cartographic_genre','Publisher','Publishing_Date','Title']) 

In [2]:
for sub in root.iter('{http://www.loc.gov/MARC21/slim}datafield'):
    print(sub.attrib,sub[0].attrib,sub[0].text,sub[1].text)

NameError: name 'root' is not defined

In [20]:
#'Author','Cartographic_genre','Publisher','Publishing_Date','Title

for sub in root.iter('{http://www.loc.gov/MARC21/slim}datafield'):
    print(sub.attrib, type(sub.attrib),sub[0].attrib,sub[0].text)
    if sub.attrib["tag"] == '110':
        auth = sub[0].text
        pub = auth
        print(auth)
    elif sub.attrib["tag"] == '655':
        cg = sub[0].text
    elif sub.attrib["tag"] == '264':
        cg = sub[0].text
    elif sub.attrib["tag"] == '655':
        cg = sub[0].text

        

{'ind1': '1', 'ind2': ' ', 'tag': '034'} <class 'dict'> {'code': 'b'} None
{'ind1': ' ', 'ind2': ' ', 'tag': '035'} <class 'dict'> {'code': 'a'} 000461
{'ind1': '2', 'ind2': ' ', 'tag': '110'} <class 'dict'> {'code': 'a'} UK Hydrographic Office
UK Hydrographic Office
{'ind1': '1', 'ind2': '0', 'tag': '245'} <class 'dict'> {'code': 'a'} Satellite Derived Coastline
{'ind1': '3', 'ind2': ' ', 'tag': '246'} <class 'dict'> {'code': 'a'} Satellite Derived Coastline
{'ind1': ' ', 'ind2': ' ', 'tag': '255'} <class 'dict'> {'code': 'b'} Transverse Mercator projection
{'ind1': ' ', 'ind2': '1', 'tag': '264'} <class 'dict'> {'code': 'a'} Taunton, UK
{'ind1': ' ', 'ind2': ' ', 'tag': '300'} <class 'dict'> {'code': 'a'} 1 geospatial database
{'ind1': '0', 'ind2': '0', 'tag': '342'} <class 'dict'> {'code': 'b'} degree
{'ind1': '0', 'ind2': '1', 'tag': '342'} <class 'dict'> {'code': 'a'} Transverse Mercator
{'ind1': '0', 'ind2': '2', 'tag': '342'} <class 'dict'> {'code': 'a'} OSGB 1936 / British Nati